In [1]:
#!import AutomatedAnalysis.dib

  Determining projects to restore...
  All projects are up-to-date for restore.
  GC.Analysis.API -> C:\Workspace\dev\performance-dev\artifacts\bin\GC.Analysis.API\Release\net8.0\GC.Analysis.API.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:01.55


Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

Loading extensions from `C:\Users\v-bibu\.nuget\packages\microsoft.data.analysis\0.19.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `C:\Users\v-bibu\.nuget\packages\xplot.plotly.interactive\4.0.7\lib\net7.0\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

In [2]:
string path = @".\Examples\Traces\All_Conserve";
DataManager dm = DataManager.CreateAspNetData(path);
string baselineConfigName = "28c";
string comparandConfigName = "28c_con1";

// Set to null if summary all benchmarks
Filter runFilter = null;
Filter configFilter = null;
List<Metric<TraceGC>> metricFilterList = ML(Metrics.G.GCPauseTime, Metrics.G.HeapSizeBefore, Metrics.G.HeapSizeAfter, Metrics.G.HeapSizePeak);
string[] benchmarkToSummary = ["Stage1Pgo", "Stage2Pgo"]; 
int[] iterationToSummary = [1, 2, 3];

## Display Benchmark Regressions

In [ ]:
Metric<BenchmarkData> avgHeapCountChanges = Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Average);
Metric<BenchmarkData> avgGCCount = Metrics.Promote(Metrics.I.TotalGCCount, Aggregation.Average);
public static Dictionary<Metric<BenchmarkData>, ExpectationDirection> HigherMemoryLowerThroughput = new()
{
    { Metrics.B.AverageMaxHeapSize, ExpectationDirection.Increase },
    { Metrics.B.AverageP50Latency, ExpectationDirection.Decrease },
    /*
    Additionally, a metric with no change is also available. The entire enum is:
    public enum ExpectationDirection
    {
        Unknown,
        Increase,
        Decrease,
        NoChange,
    }
    */
};

Dictionary< Metric<BenchmarkData>, double> thresholds = new()
{
    { Metrics.B.AverageMaxHeapSize, 5 },
    { Metrics.B.AverageP50Latency, 5 },
};

DisplayBenchmarkSummaryWithRegressions(dm, baselineConfigName, comparandConfigName, metrics: ML( Metrics.B.AverageMaxHeapSize, Metrics.B.AverageP50Latency ), benchmarkRegressionThreshold: 5, benchmarksToDisplay: 5, 
metricToleranceOverridePercentage: thresholds,
expectationDirections: HigherMemoryLowerThroughput);

## Display Iteration Anomalies

In [ ]:
// Update Average -> Safe Average.
Aggregation SafeAverage = new Aggregation((gc => {
    double safeAverage;
    
    long count = 0;
    double sum = 0;

    foreach (var item in gc)
    {
        if (item != 0 && !double.IsNaN(item))
        {
            sum += item;
            count++;
        }
    }

    safeAverage = count == 0 ? double.NaN : sum / count;
    return safeAverage;

}), "SafeAverage", "");

Metric<IterationData> avgTCPToConsider = Metrics.Promote(Metrics.G.MedianThroughputCostPercent, SafeAverage);

Dictionary<Metric<IterationData>, double> volatilityOverrides = new()
{
    { Metrics.I.Gen0MeanPauseMSec, 10 },
    { Metrics.I.Latency50thMS, 20 },
};

// todo: override the volatility threshold
DisplayIterationData(dm, baselineConfigName, comparandConfigName, metrics: ML( avgTCPToConsider, Metrics.I.Gen0MeanPauseMSec, Metrics.I.Latency50thMS ) //, benchmarkFilter: new Filter("Stage1Pgo"),
,volatilityThreshold: 2, metricVolOverridePercentage: volatilityOverrides, iterationsToDisplay: 5) 

## Display GC data

In [3]:
foreach (var metricFilter in metricFilterList)
{
    foreach (var benchmarkName in benchmarkToSummary)
    {
        Filter benchmarkFilter = new(benchmarkName);
        IntFilter iterationFilter = new(iterationToSummary);

        ChartGCData(dm, metricFilter, benchmarkFilter: benchmarkFilter, iterationFilter: iterationFilter)
        .ForEach(ch => ch.Display());
        
        StringBuilder SummaryTableBuilder = new();
        SummaryTableBuilder.AppendLine($"## Summary and details for {metricFilter.Title} of {benchmarkName}");
        SummaryTableBuilder.AppendLine($"| Run | Avg. {metricFilter.Title}({metricFilter.Unit}) |");
        SummaryTableBuilder.AppendLine("| -------- | --------: |");

        Dictionary<string, RunData> rdDict = runFilter == null ?
            dm.Data.Runs : dm.Data.Runs.Where(run => runFilter.Include(run.Key)).ToDictionary();

        foreach (var runName in rdDict.Keys)
        {
            RunData rd = rdDict[runName];
            Dictionary<string, ConfigData> cdDict = configFilter == null ?
                rd.Configs : rd.Configs.Where(config => configFilter.Include(config.Key)).ToDictionary();

            foreach (var configName in cdDict.Keys)
            {
                ConfigData cd = rd.Configs[configName];
                Dictionary<string, BenchmarkData> bdDict = benchmarkFilter == null ?
                    cd.Benchmarks : cd.Benchmarks.Where(benchmark => benchmarkFilter.Include(benchmark.Key)).ToDictionary();

                BenchmarkData bd = cd.Benchmarks[benchmarkName];

                List<(IterationData, int)> iterationWithIndexList = iterationFilter == null ?
                    bd.Iterations.WithIndex().ToList() : 
                    bd.Iterations.WithIndex().Where((_, idx) => iterationFilter.Include(idx)).ToList();
                foreach (var (iterationData, iterationIdx) in iterationWithIndexList)    
                {
                    double data = iterationData.GCProcessData.GCs
                        .Select(gc => metricFilter.DoExtract(gc, 0))
                        .Average().Value;
                    SummaryTableBuilder.AppendLine(
                        $"| {configName}_{iterationIdx} | {data.ToString("f2")} |");
                }
                
            }
        }

        SummaryTableBuilder.ToString().DisplayAs("text/markdown");
    }
}

<!DOCTYPE html>

<!DOCTYPE html>

## Summary and details for GCPauseTime of Stage1Pgo
| Run | Avg. GCPauseTime(ms) |
| -------- | --------: |
| 28c_1 | 1150.68 |
| 28c_2 | 1054.93 |
| 28c_3 | 1061.85 |
| 28c_con1_1 | 1041.98 |
| 28c_con1_2 | 944.19 |
| 28c_con1_3 | 1161.19 |


<!DOCTYPE html>

<!DOCTYPE html>

...

<!DOCTYPE html>

## Summary and details for Heap size (peak) of Stage1Pgo
| Run | Avg. Heap size (peak)(MB) |
| -------- | --------: |
| 28c_1 | 23.05 |
| 28c_2 | 20.35 |
| 28c_3 | 19.13 |
| 28c_con1_1 | 20.44 |
| 28c_con1_2 | 19.13 |
| 28c_con1_3 | 21.51 |


<!DOCTYPE html>

<!DOCTYPE html>

## Summary and details for Heap size (peak) of Stage2Pgo
| Run | Avg. Heap size (peak)(MB) |
| -------- | --------: |
| 28c_1 | 75.42 |
| 28c_2 | 68.57 |
| 28c_3 | 64.00 |
| 28c_con1_1 | 84.00 |
| 28c_con1_2 | 88.60 |
| 28c_con1_3 | 91.75 |
